dewiuqdewideiugeago


## goals: 

1. class imbalance:
	* Y  (copy the entire row including X, add noise to X) 
    * 拼3年的数据，让Y=1的数据～150， Y=0～150，选出共同的X 
    * 拼成一个大表：有x有y
2. missing value: Y - drop NaN, X - use mode for now
3. model: classification - logistic regression
4. features X: select all related variables from demo & questionnaire, general (age, income, gender, … )
 

## Import modules and libraries

In [239]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
from sklearn.preprocessing import Imputer
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing 
from fancyimpute import KNN   
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn import metrics


%matplotlib inline

## Load data for Y

In [240]:
# Y: CIDDSCOR - depression score, target: Both males and females 20 YEARS - 39 YEARS
# Can't impute Y, missing Y must be dropped
# SEQN are built upon previous years, can vertically stack 3 depression datasets

df_depression1 = pd.read_sas('DEPRESSION/CIQMDEP.XPT')[['SEQN','CIDDSCOR']].dropna()  # 1:positive-50, 5:negative-664, missing-64
df_depression2 = pd.read_sas('DEPRESSION/CIQDEP_B.XPT')[['SEQN','CIDDSCOR']].dropna() # 1:positive-48, 5:negative-764, missing-116
df_depression3 = pd.read_sas('DEPRESSION/CIQDEP_C.XPT')[['SEQN','CIDDSCOR']].dropna() # 1:positive-50, 5:negative-641, missing-159
dfs_depression = pd.concat([df_depression1, df_depression2, df_depression3])
dfs_depression


,SEQN,CIDDSCOR
0,12.0,5.0
1,20.0,5.0
2,34.0,5.0
3,66.0,5.0
4,69.0,5.0
5,81.0,5.0
6,97.0,5.0
7,107.0,5.0
8,120.0,5.0
9,143.0,5.0


## Load data for X

In [241]:
# CODE	QUESTION	VALUES
# RIAGENDR	gender	1,2
# RIDAGEYR	Age at Screening Adjudicated	0-84, 85
# RIDRETH1	Race/Ethnicity	1,2,3,4,5

df_demo1 = pd.read_sas('DEMO/DEMO.XPT')[['SEQN','RIAGENDR','RIDAGEYR','RIDRETH1']]   # 1:positive-50, 5:negative-664, missing-64
df_demo2 = pd.read_sas('DEMO/DEMO_B.XPT')[['SEQN','RIAGENDR','RIDAGEYR','RIDRETH1']] # 1:positive-48, 5:negative-764, missing-116
df_demo3 = pd.read_sas('DEMO/DEMO_C.XPT')[['SEQN','RIAGENDR','RIDAGEYR','RIDRETH1']] # 1:positive-50, 5:negative-641, missing-159
dfs_demo = pd.concat([df_demo1, df_demo2, df_demo3])
dfs_demo

,SEQN,RIAGENDR,RIDAGEYR,RIDRETH1
0,1.0,2.0,2.000000e+00,4.0
1,2.0,1.0,7.700000e+01,3.0
2,3.0,2.0,1.000000e+01,3.0
3,4.0,1.0,1.000000e+00,4.0
4,5.0,1.0,4.900000e+01,3.0
5,6.0,2.0,1.900000e+01,5.0
6,7.0,2.0,5.900000e+01,4.0
7,8.0,1.0,1.300000e+01,3.0
8,9.0,2.0,1.100000e+01,4.0
9,10.0,1.0,4.300000e+01,4.0


## Zeros, constraints(no response), initialize dataframe

In [242]:
def clean_epsilon(cell):
    if cell - 0.0 < 0.0001:
        return 0
    return cell

def constraints(cell, cstr):
    # forbidden values
    if cell in cstr: return np.nan
    else: return cell


df = pd.merge(dfs_depression, dfs_demo, on='SEQN')
df

# # initialize the dataframe
# prev_df = dfs_depression
# # merge dfs, turn min values into zeros, and get rid of don't know & no response values
# def merge_and_clean(new_feature, prev_df, cstr):
#     dataset_name = list(new_feature)[0]
#     dataset_col = list(new_feature.values())[0]
#     cur_df = pd.read_sas(dataset_name)[['SEQN', dataset_col]]
#     cur_df[dataset_col] = cur_df[dataset_col].apply(clean_epsilon)
#     cur_df[dataset_col] = cur_df[dataset_col].apply(lambda x: constraints(x,cstr))
#     df = pd.merge(prev_df, cur_df, on='SEQN')
#     prev_df = df 
#     return prev_df

,SEQN,CIDDSCOR,RIAGENDR,RIDAGEYR,RIDRETH1
0,12.0,5.0,1.0,37.0,3.0
1,20.0,5.0,2.0,23.0,1.0
2,34.0,5.0,2.0,38.0,4.0
3,66.0,5.0,1.0,37.0,1.0
4,69.0,5.0,1.0,27.0,4.0
5,81.0,5.0,1.0,30.0,3.0
6,97.0,5.0,1.0,32.0,4.0
7,107.0,5.0,2.0,30.0,1.0
8,120.0,5.0,2.0,30.0,3.0
9,143.0,5.0,2.0,22.0,1.0


## Dealing with missing values

In [243]:
def treat_nan_as_another_category(df):
    print('Missing values: treat as another category')
    df = df.fillna(50)
    return df

def impuate_nan_as_mode(df):
    print('Missing values: impute using mode')
    my_imputer = Imputer(missing_values='NaN', strategy='most_frequent')
    data_with_imputed_values = my_imputer.fit_transform(df) # type: 2D array
    data_with_imputed_values = pd.DataFrame(data_with_imputed_values) # change 2D array to dataframe
    data_with_imputed_values.columns = df.columns # give the newly created dataframe column names 
    df = data_with_imputed_values
    return df

def drop_nan(df):
    print('Missing values: rows with NaN are dropped')
    df = df.dropna()
    return df


'''Now we have three variables, the gender, the income and the level of depression that has missing values. 
We then assume that people of similar income and of same gender tend to have the same level of depression. 
For a given missing value, we will look at the gender of the person, its income, look for its k nearest 
neighbors and get their level of depression. We can then approximate the depression level of the person we wanted.
knn explained: https://www.dataquest.io/blog/k-nearest-neighbors-in-python/ '''


X1 = [[0], [1], [2], [3]] # X must be 2D array
y1 = [0, 0, 1, 1]

def impute_nan_using_KNN(X,y):    

    neigh = KNeighborsClassifier(n_neighbors=3)
    neigh.fit(X, y) 
    print(neigh.predict([[2.5]]))



## data standardization

## Model

In [245]:
def LinearSVC(X_train, X_test, y_train, y_test):
    print('Model: svm.LinearSVC.')
    clf = svm.LinearSVC(C=1000,dual=False)
    clf.fit(X_train, y_train)
    train_predictions = clf.predict(X_train)
    test_predictions = clf.predict(X_test)
    return train_predictions, test_predictions

def LogisticRegressionModel(X_train, X_test, y_train, y_test):
    print('Model: Logistic Regression')
    lr = LogisticRegression(C=1,dual=False)
    lr.fit(X_train, y_train)
    train_predictions = lr.predict(X_train)
    test_predictions = lr.predict(X_test)
    return train_predictions, test_predictions

## RUN

In [246]:
def compute_accuracy(predict, actual):
        if len(predict) != len(actual):
            return 0
        correctness = 0
        for i in range(len(predict)):
            if predict[i] == actual[i]: 
                correctness += 1
            i += 1
        accuracy = correctness / len(predict)
        return accuracy
    
def run_prediction(df):

# pick one of the methods here to deal with missing values 
##########################################################
    # impute NaN values as mode of each col
    df = impuate_nan_as_mode(df)
    # treat NaN as a new category
    # df = treat_nan_as_another_category(df)
    # drop rows that contain NaN
    # df = drop_nan(df)
##########################################################

    y = df['CIDDSCOR'].values
    columns_to_delete = ['SEQN', 'CIDDSCOR']
    remaining_df = df.drop(columns=columns_to_delete)
    X = remaining_df.values

    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)
    train_predictions, test_predictions = LinearSVC(X_train, X_test, y_train, y_test)
    #train_predictions, test_predictions = LogisticRegressionModel(X_train, X_test, y_train, y_test)

    print('Current features are:', remaining_df.columns.values)
    print('The training set accuracy is', compute_accuracy(train_predictions, y_train)*100,'%')
    print('The testing set accuracy is', compute_accuracy(test_predictions, y_test)*100,'%')
    print('------------------------------------------------------')

run_prediction(df)

Missing values: impute using mode
Model: svm.LinearSVC.
Current features are: ['RIAGENDR' 'RIDAGEYR' 'RIDRETH1']
The training set accuracy is 93.00620417371687 %
The testing set accuracy is 94.5945945945946 %
------------------------------------------------------


- LinearSVC

        Missing values: impute using mode
        Model: svm.LinearSVC.
        Current features are: ['RIAGENDR' 'RIDAGEYR' 'RIDRETH1']
        The training set accuracy is 93.1190073322053 %
        The testing set accuracy is 94.14414414414415 %
        
- LogisticRegression
    
        Missing values: impute using mode
        Model: Logistic Regression
        Current features are: ['RIAGENDR' 'RIDAGEYR' 'RIDRETH1']
        The training set accuracy is 93.79582628313592 %
        The testing set accuracy is 91.44144144144144 %

# Questions:
- 
- smote - class imbalance